In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
spark1 = SparkSession.builder.appName('test').getOrCreate()

In [3]:
#Create a Spark Session
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("test") \
    .config("spark.executor.memory", "0.1g") \
    .config("spark.cores.max","2") \
    .config("spark.sql.warehouse.dir", "/home/sushant/Projects/Spark_Project/temp")\
    .getOrCreate()
    
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

#Test Spark
testData = SpContext.parallelize([3,6,4,2])
testData.count()
#check http://localhost:4040 to see if Spark is running


4

In [10]:
#Spark has a lazy execution
tweetsRDD = SpContext.textFile('movietweets.csv')
display(tweetsRDD.take(5))
ucrRDD = tweetsRDD.map(lambda x: x.upper())
display(ucrRDD.take(5))
tweetsRDD.count()
tweetsRDD.collect() # only for small dataset
tweetsRDD.first()
autoData = SpContext.textFile("auto-data.csv")
autoData.cache()
print autoData.take(5)
autoData.count()
#for line in autoData.collect():
    #print line
# Save loaded data to normal file
autoDataFile = open("auto-Data-save.csv","w")
autoDataFile.write('\n'.join(autoData.collect()))
autoDataFile.close()

[u'positive,The Da Vinci Code book is just awesome.',
 u'positive,i liked the Da Vinci Code a lot.',
 u'positive,i liked the Da Vinci Code a lot.',
 u"positive,I liked the Da Vinci Code but it ultimatly didn't seem to hold it's own.",
 u"positive,that's not even an exaggeration ) and at midnight we went to Wal-Mart to buy the Da Vinci Code"]

[u'POSITIVE,THE DA VINCI CODE BOOK IS JUST AWESOME.',
 u'POSITIVE,I LIKED THE DA VINCI CODE A LOT.',
 u'POSITIVE,I LIKED THE DA VINCI CODE A LOT.',
 u"POSITIVE,I LIKED THE DA VINCI CODE BUT IT ULTIMATLY DIDN'T SEEM TO HOLD IT'S OWN.",
 u"POSITIVE,THAT'S NOT EVEN AN EXAGGERATION ) AND AT MIDNIGHT WE WENT TO WAL-MART TO BUY THE DA VINCI CODE"]

u'positive,The Da Vinci Code book is just awesome.'

In [25]:
#Transformation
tsvData = autoData.map(lambda x: x.replace(",","\t"))
tsvData.take(5)
toyotaData = autoData.filter(lambda x: "toyota" in x)
toyotaData.count()
words = toyotaData.flatMap(lambda line : line.split(','))
words.take(20)
words.count()
for x in testData.distinct().collect():
    print x
word1 = SpContext.parallelize(['hey','where','are','you'])
word2 = SpContext.parallelize(['I', 'hey','am','coming'])
for intersects in word1.union(word2).distinct().collect():
    print intersects
for intersects in word1.intersection(word2).distinct().collect():
    print intersects

#pass a function to be worked on whole data
def cleansingRDD(autostr):
    if isinstance(autostr,int):
        return autostr
    attlist = autostr.split(",")
    attlist[5] = attlist[5].upper()
    return ",".join(attlist)
    
cleanedRDD = autoData.map(cleansingRDD)
cleanedRDD.collect()

384

In [41]:
#reduce operations on RDD
testData.collect()
testData.reduce(lambda x, y: x+y)
autoData.reduce(lambda x,y : y if len(x)> len(y) else x)

#Find average MPG for all cars

def getMPG(autostr):
    if isinstance(autostr, int):
        return autostr
    attlist = autostr.split(",")
    if attlist[9].isdigit():
        return int(attlist[9])
    else:
        return 0
autoData.reduce(lambda x,y : getMPG(x)+getMPG(y))/(autoData.count()-1) # -1 to reduce the header row

25

In [57]:
#Keyvalue pair
kpairData = autoData.map(lambda x:(x.split(',')[0],x.split(',')[7]))
kpairData.take(5)
kpairData.keys().collect()
header = kpairData.first()
kpairData = kpairData.filter(lambda line: line !=header)

# Find average mpg for every brand
addOne = kpairData.mapValues(lambda x : (x,1))
brandValues = addOne.reduceByKey(lambda x,y:(int(x[0])+int(y[0]),x[1]+y[1]))
averageBrand = brandValues.mapValues(lambda x: int(x[0])/int(x[1])).collect()

In [64]:
# Accumulator, broadcast, persistance of variables

sedanCount = SpContext.accumulator(0)
hatchbackCount = SpContext.accumulator(0)

sedanText = SpContext.broadcast("sedan")
hatchbackText = SpContext.broadcast("hatchback")

def splitlines(line):
    global sedanCount
    global hatchbackCount
    
    if sedanText.value in line:
        sedanCount += 1
    elif hatchbackText.value in line:
        hatchbackCount += 1
        
    return line.split(',')

autoTypeCount = autoData.map(splitlines)
autoTypeCount.count()
print hatchbackCount, sedanCount

testData.getNumPartitions()

newRDD = SpContext.parallelize([1,2,3,4,5],3)
newRDD.cache()
newRDD.collect()
newRDD.getNumPartitions()

67 92


3